# Solution: Week 5 - Logical Agents

## Overview

Complete this assignment by completing the steps below.  The assignment has two parts, a base-portion designed to allow you to practice applying search to a new set of data, and a bonus portion where you can challenge yourself and potentially earn additional points.

1. Configure your environment

1. Study the algorithms and read the code

1. Run through the example exercises using the undirected graph of Romanian roads

1. Apply what you have learned to a map of Colorado roads

1. Answer the questions posed at the end of this document

1. Save your notebook as a PDF and submit it to the Assignments on Worldclass

## Introduction to Logic Notebook

This notebook describes the [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) module, which covers Chapters 6 (Logical Agents),  7 (First-Order Logic) and  8 (Inference in First-Order Logic) of *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

We'll start by looking at `Expr`, the data type for logical sentences, and the convenience function `expr`. We'll be covering two types of knowledge bases, `PropKB` - Propositional logic knowledge base and `FolKB` - First order logic knowledge base. We will construct a propositional knowledge base of a specific situation in the Wumpus World. We will next go through the `tt_entails` function and experiment with it a bit. The `pl_resolution` and `pl_fc_entails` functions will come next. We'll study forward chaining and backward chaining algorithms for `FolKB` and use them on `crime_kb` knowledge base.

But the first step is to load the code:

In [1]:
from utils import *
from logic import *
from notebook import psource

## Logical Sentences

The `Expr` class is designed to represent any kind of mathematical expression. The simplest type of `Expr` is a symbol, which can be defined with the function `Symbol`:

In [2]:
Symbol('x')

x

Or we can define multiple symbols at the same time with the function `symbols`:

In [3]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

We can combine `Expr`s with the regular Python infix and prefix operators. Here's how we would form the logical sentence "P and not Q":

In [4]:
P & ~Q

(P & ~Q)

This works because the `Expr` class overloads the `&` operator with this definition:

```python
def __and__(self, other): return Expr('&',  self, other)```
     
and does similar overloads for the other operators. An `Expr` has two fields: `op` for the operator, which is always a string, and `args` for the arguments, which is a tuple of 0 or more expressions. By "expression," I mean either an instance of `Expr`, or a number. Let's take a look at the fields for some `Expr` examples:

In [5]:
sentence = P & ~Q

sentence.op

'&'

In [6]:
sentence.args

(P, ~Q)

In [7]:
P.op

'P'

In [8]:
P.args

()

In [9]:
Pxy = P(x, y)

Pxy.op

'P'

In [10]:
Pxy.args

(x, y)

It is important to note that the `Expr` class does not define the *logic* of Propositional Logic sentences; it just gives you a way to *represent* expressions. Think of an `Expr` as an [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree).  Each of the `args` in an `Expr` can be either a symbol, a number, or a nested `Expr`. We can nest these trees to any depth. Here is a deply nested `Expr`:

In [11]:
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

## Operators for Constructing Logical Sentences

Here is a table of the operators that can be used to form sentences. Note that we have a problem: we want to use Python operators to make sentences, so that our programs (and our interactive sessions like the one here) will show simple code. But Python does not allow implication arrows as operators, so for now we have to use a more verbose notation that Python does allow: `|'==>'|` instead of just `==>`. Alternately, you can always use the more verbose `Expr` constructor forms:

| Operation                | Book | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

Here's an example of defining a sentence with an implication arrow:

In [12]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

## `expr`: a Shortcut for Constructing Sentences

If the `|'==>'|` notation looks ugly to you, you can use the function `expr` instead:

In [13]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

`expr` takes a string as input, and parses it into an `Expr`. The string can contain arrow operators: `==>`, `<==`, or `<=>`, which are handled as if they were regular Python infix operators. And `expr` automatically defines any symbols, so you don't need to pre-define them:

In [14]:
expr('sqrt(b ** 2 - 4 * a * c)')

sqrt(((b ** 2) - ((4 * a) * c)))

For now that's all you need to know about `expr`. If you are interested, we explain the messy details of how `expr` is implemented and how `|'==>'|` is handled in the appendix.

## Propositional Knowledge Bases: `PropKB`

The class `PropKB` can be used to represent a knowledge base of propositional logic sentences.

We see that the class `KB` has four methods, apart from `__init__`. A point to note here: the `ask` method simply calls the `ask_generator` method. Thus, this one has already been implemented, and what you'll have to actually implement when you create your own knowledge base class (though you'll probably never need to, considering the ones we've created for you) will be the `ask_generator` function and not the `ask` function itself.

The class `PropKB` now.
* `__init__(self, sentence=None)` : The constructor `__init__` creates a single field `clauses` which will be a list of all the sentences of the knowledge base. Note that each one of these sentences will be a 'clause' i.e. a sentence which is made up of only literals and `or`s.
* `tell(self, sentence)` : When you want to add a sentence to the KB, you use the `tell` method. This method takes a sentence, converts it to its CNF, extracts all the clauses, and adds all these clauses to the `clauses` field. So, you need not worry about `tell`ing only clauses to the knowledge base. You can `tell` the knowledge base a sentence in any form that you wish; converting it to CNF and adding the resulting clauses will be handled by the `tell` method.
* `ask_generator(self, query)` : The `ask_generator` function is used by the `ask` function. It calls the `tt_entails` function, which in turn returns `True` if the knowledge base entails query and `False` otherwise. The `ask_generator` itself returns an empty dict `{}` if the knowledge base entails query and `None` otherwise. This might seem a little bit weird to you. After all, it makes more sense just to return a `True` or a `False` instead of the `{}` or `None` But this is done to maintain consistency with the way things are in First-Order Logic, where an `ask_generator` function is supposed to return all the substitutions that make the query true. Hence the dict, to return all these substitutions. I will be mostly be using the `ask` function which returns a `{}` or a `False`, but if you don't like this, you can always use the `ask_if_true` function which returns a `True` or a `False`.
* `retract(self, sentence)` : This function removes all the clauses of the sentence given, from the knowledge base. Like the `tell` function, you don't have to pass clauses to remove them from the knowledge base; any sentence will do fine. The function will take care of converting that sentence to clauses and then remove those.

## Wumpus World KB
Let us create a `PropKB` for the wumpus world with the sentences mentioned in `section 7.4.3`.

In [15]:
wumpus_kb = PropKB()

We define the symbols we use in our clauses.<br/>
$P_{x, y}$ is true if there is a pit in `[x, y]`.<br/>
$B_{x, y}$ is true if the agent senses breeze in `[x, y]`.<br/>

In [16]:
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

Now we tell sentences based on `section 7.4.3`.<br/>
There is no pit in `[1,1]`.

In [17]:
wumpus_kb.tell(~P11)

A square is breezy if and only if there is a pit in a neighboring square. This has to be stated for each square but for now, we include just the relevant squares.

In [18]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

Now we include the breeze percepts for the first two squares leading up to the situation in `Figure 7.3(b)`

In [19]:
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

We can check the clauses stored in a `KB` by accessing its `clauses` variable

In [20]:
wumpus_kb.clauses

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

We see that the equivalence $B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was automatically converted to two implications which were inturn converted to CNF which is stored in the `KB`.<br/>
$B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was split into $B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ and $B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$.<br/>
$B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ was converted to $P_{1, 2} \lor P_{2, 1} \lor \neg B_{1, 1}$.<br/>
$B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$ was converted to $\neg (P_{1, 2} \lor P_{2, 1}) \lor B_{1, 1}$ which becomes $(\neg P_{1, 2} \lor B_{1, 1}) \land (\neg P_{2, 1} \lor B_{1, 1})$ after applying De Morgan's laws and distributing the disjunction.<br/>
$B_{2, 1} \iff (P_{1, 1} \lor P_{2, 2} \lor P_{3, 2})$ is converted in similar manner.

## Inference in Propositional Knowledge Base
In this section we will look at two algorithms to check if a sentence is entailed by the `KB`. Our goal is to decide whether $\text{KB} \vDash \alpha$ for some sentence $\alpha$.
### Truth Table Enumeration
It is a model-checking approach which, as the name suggests, enumerates all possible models in which the `KB` is true and checks if $\alpha$ is also true in these models. We list the $n$ symbols in the `KB` and enumerate the $2^{n}$ models in a depth-first manner and check the truth of `KB` and $\alpha$.

In [21]:
psource(tt_check_all)

The algorithm basically computes every line of the truth table $KB\implies \alpha$ and checks if it is true everywhere.
<br>
If symbols are defined, the routine recursively constructs every combination of truth values for the symbols and then, 
it checks whether `model` is consistent with `kb`.
The given models correspond to the lines in the truth table,
which have a `true` in the KB column, 
and for these lines it checks whether the query evaluates to true
<br>
`result = pl_true(alpha, model)`.
<br>
<br>
In short, `tt_check_all` evaluates this logical expression for each `model`
<br>
`pl_true(kb, model) => pl_true(alpha, model)`
<br>
which is logically equivalent to
<br>
`pl_true(kb, model) & ~pl_true(alpha, model)` 
<br>
that is, the knowledge base and the negation of the query are logically inconsistent.
<br>
<br>
`tt_entails()` just extracts the symbols from the query and calls `tt_check_all()` with the proper parameters.


In [22]:
psource(tt_entails)

Keep in mind that for two symbols P and Q, P => Q is false only when P is `True` and Q is `False`.
Example usage of `tt_entails()`:

In [23]:
tt_entails(P & Q, Q)

True

P & Q is True only when both P and Q are True. Hence, (P & Q) => Q is True

In [24]:
tt_entails(P | Q, Q)

False

In [25]:
tt_entails(P | Q, P)

False

If we know that P | Q is true, we cannot infer the truth values of P and Q. 
Hence (P | Q) => Q is False and so is (P | Q) => P.

In [26]:
(A, B, C, D, E, F, G) = symbols('A, B, C, D, E, F, G')
tt_entails(A & (B | C) & D & E & ~(F | G), A & D & E & ~F & ~G)

True

We can see that for the KB to be true, A, D, E have to be True and F and G have to be False.
Nothing can be said about B or C.

Coming back to our problem, note that `tt_entails()` takes an `Expr` which is a conjunction of clauses as the input instead of the `KB` itself. 
You can use the `ask_if_true()` method of `PropKB` which does all the required conversions. 
Let's check what `wumpus_kb` tells us about $P_{1, 1}$.

In [27]:
wumpus_kb.ask_if_true(~P11), wumpus_kb.ask_if_true(P11)

(True, False)

Looking at Figure 7.9 we see that in all models in which the knowledge base is `True`, $P_{1, 1}$ is `False`. It makes sense that `ask_if_true()` returns `True` for $\alpha = \neg P_{1, 1}$ and `False` for $\alpha = P_{1, 1}$. This begs the question, what if $\alpha$ is `True` in only a portion of all models. Do we return `True` or `False`? This doesn't rule out the possibility of $\alpha$ being `True` but it is not entailed by the `KB` so we return `False` in such cases. We can see this is the case for $P_{2, 2}$ and $P_{3, 1}$.

In [28]:
wumpus_kb.ask_if_true(~P22), wumpus_kb.ask_if_true(P22)

(False, False)

# Begin Assignment ---------------------------

## Managing Your Working Directory

To use the code from aima-python, you need to be running your notebook or python code from root directory of the repository you just created.  _You will need to update the location of aima-python on your system._

In [ ]:
# jupyter started here
%pwd

In [1]:
# UPDATE TO POINT TO THE LOCATION OF THE AIMA-PYTHON REPOSITORY ON YOUR SYSTEM
%cd /Users/rob/Documents/GitHub

/Users/rob/Documents/GitHub


In [2]:
# You should now be in the repository
%pwd

'/Users/rob/Documents/GitHub'

Note: aima-python is _not_ a python package which means that you cannot conda or pip install it.  That is why you need to be in the repository in order to make use of repositories' code.  

## Representing Knowledge in Python

Refer to the Wumpus World example (in cells 15 to 20 above) to help you solve problems 1 and 2.  Create a knowledge base using the facts stated in the problem and the `tell()` function.  Represent the questions you need to answer as sentences and evaluate them using `ask-if-true()`. 

### Question 1: 
Given the following, can you prove that the unicorn is mythical? How about magical? Horned?  If the unicorn is mythical, then it is immortal, but if it is not mythical, then it is a mortal mammal.  If the unicorn is either immortal or a mammal, then it is horned.  The unicorn is magical if it is horned.  

_Using the example for Wumpus World above, solve problem 7.2 using the `tell` and `ask-if-true`._

In [2]:
# Y0UR CODE HERE / ADD EXTRA CELLS AS NEEDED

Do you think the answer is correct?  Why or why not?  

--- YOUR ANSWER HERE ---

How could you test it?  

--- YOUR ANSWER HERE ---

### Question 2
Suppose the agent has progressed to the point shown in Figure 7.4(a), page 239, having perceived nothing in [1,1], a breeze in [2,1], and a stench in [1,2], and is now concerned with the contents of [1,3], [2,2], and [3,1]. Each of these can contain a pit, and at most one can contain a wumpus.  

Show that logical agents for each of the following sentences are true:

* α2 = 'There is no pit in [2,2].'

* α3 = 'There is a wumpus in [1,3].'

* Hence show that KB |= α2 and KB |= α3.

_Using the example in the Inference in Propositional Knowledge Base section above, solve this problem using the `tt_entails()`._

In [3]:
# Y0UR CODE HERE / ADD EXTRA CELLS AS NEEDED

Do you think the answer is correct?  Why or why not?  

--- YOUR ANSWER HERE ---

How could you test it?  

--- YOUR ANSWER HERE ---

# End assignment ---------------------------

## Appendix: The Implementation of `|'==>'|`

Consider the `Expr` formed by this syntax:

In [90]:
P |'==>'| ~Q

(P ==> ~Q)

What is the funny `|'==>'|` syntax? The trick is that "`|`" is just the regular Python or-operator, and so is exactly equivalent to this: 

In [91]:
(P | '==>') | ~Q

(P ==> ~Q)

In other words, there are two applications of or-operators. Here's the first one:

In [92]:
P | '==>'

PartialExpr('==>', P)

What is going on here is that the `__or__` method of `Expr` serves a dual purpose. If the right-hand-side is another `Expr` (or a number), then the result is an `Expr`, as in `(P | Q)`. But if the right-hand-side is a string, then the string is taken to be an operator, and we create a node in the abstract syntax tree corresponding to a partially-filled  `Expr`, one where we know the left-hand-side is `P` and the operator is `==>`, but we don't yet know the right-hand-side.

The `PartialExpr` class has an `__or__` method that says to create an `Expr` node with the right-hand-side filled in. Here we can see the combination of the `PartialExpr` with `Q` to create a complete `Expr`:

In [93]:
partial = PartialExpr('==>', P) 
partial | ~Q

(P ==> ~Q)

This  [trick](http://code.activestate.com/recipes/384122-infix-operators/) is due to [Ferdinand Jamitzky](http://code.activestate.com/recipes/users/98863/), with a modification by [C. G. Vedant](https://github.com/Chipe1),
who suggested using a string inside the or-bars.

## Appendix: The Implementation of `expr`

How does `expr` parse a string into an `Expr`? It turns out there are two tricks (besides the Jamitzky/Vedant trick):

1. We do a string substitution, replacing "`==>`" with "`|'==>'|`" (and likewise for other operators).
2. We `eval` the resulting string in an environment in which every identifier
is bound to a symbol with that identifier as the `op`.

In other words,

In [94]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

is equivalent to doing:

In [95]:
P, Q = symbols('P, Q')
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

One thing to beware of: this puts `==>` at the same precedence level as `"|"`, which is not quite right. For example, we get this:

In [96]:
P & Q  |'==>'|  P | Q

(((P & Q) ==> P) | Q)

which is probably not what we meant; when in doubt, put in extra parens:

In [97]:
(P & Q)  |'==>'|  (P | Q)

((P & Q) ==> (P | Q))

# Authors

This notebook was originally created by [Chirag Vartak](https://github.com/chiragvartak) and [Peter Norvig](https://github.com/norvig) and has been adapted by Rob Osterburg for MSDS688.  
